In [1]:
import numpy as np
import random

In [2]:
TETMAX = 100    # объем ресурса
TET = TETMAX    # объем свободного ресурса
mintet = []     # список минимальных запращиваемых объемов ресурса соответствующими заявками
Time = 0        # счетчик модельного времени
TimeLim = 1000  # ограничение времени симуляции
lamb = 10       # параметр экспоненциального распределения времени меджу поступлениями заявок
mu = 10         # параметр экспоненциального распределения "размера" заявки
tau = 1         # параметр экспоненциального распределения времени ожидания в очереди
Rmax = 5        # кол-во мест в очереди
R = []          # очередь
Mu = []         # список прогнозируемых моментов окончания обслуживания соответсвующих заявок
WS = []         # список для записи момента поступления заявок в очередь
TetR = []       # аналог mintet для заявок в очереди
All = 0         # счетчик всех поступивших заявок
STP = 0         # сумма времен обслуживания заявок (для вычисления средней продолжительности обслуживания)
SWT = 0         # сумма времен ожиданий в очереди (для вычисления среднего ожидания)
S3 = 0          # для вычисления среднего числа заявок на приборах
SR = 0          # для вычисления средней длины очереди
Nbg = 0         # переменная для записи количества заявок на приборах в момент времени
block1 = 0      # счетчик заявок, не дождавшихся поступления на обслуживание
block2 = 0      # счетчик отклоненных заявок

In [3]:
while Time<TimeLim:
    dt = random.expovariate(lamb)
    Time += dt
    All += 1
    
    while any(x < Time for x in Mu):  # проверяем вышли ли заявки за этот интервал
        I = np.argmin(Mu)
        Nbg -= 1
        w = Mu.pop(I)
        TET += mintet.pop(I)
        j=0
        
        for k in range(len(R)):
            if R[j]<w:
                # заявка не дождалась
                R.pop(j)
                block1 += 1
            else:
                # заявка дождалась
                j += 1
        
        if len(R)!=0:
            j=0
            while TET>TetR[0]:
                j += 1
                Nbg += 1
                R.pop(0)
                SWT += w - WS.pop(0)
                mintet.append(TetR.pop(0))
                TET -= mintet[-1]
                Mu.append(w + random.expovariate(mu)*TETMAX/(mintet[-1]+TET/Nbg)) #, то она поступает на прибор
                STP += Mu[-1] - w
                
                if len(R)==0:
                    break
        
        for i in range(Nbg-j):
            Mu[i] = w + (Mu[i]-w)*(mintet[i]+TET/(Nbg-j+1))/(mintet[i]+TET/Nbg)
    
    S3 += Nbg
    SR += len(R)
    
    # поступление новой заявки:
    mt = np.random.uniform(8,15)  # минимальный требуемый объём ресурса для новой заявки
    if TET > mt:
        # ресурса хватает для немедленного поступления на обработку
        Nbg += 1
        
        # ресурс перераспределяется и необходимо пересчитать прогнозируемые моменты окончания
        for i in range(Nbg-1):
            Mu[i] = Time + (Mu[i]-Time)*(mintet[i] + TET/(Nbg-1))/(mintet[i] + TET/Nbg)
        
        mintet.append(mt)
        TET -= mt
        Mu.append(Time+random.expovariate(mu)*TETMAX/(mintet[-1]+TET/Nbg)) 
        # экспоненциально с параметром mu распределено минимальное время обслуживания заявки, то есть если бы она заняла весь ресурс
        
        STP += Mu[-1] - Time
    
    elif len(R)<Rmax:
        # в очереди есть место
        R.append(Time+random.expovariate(tau))
        WS.append(Time)
        TetR.append(mt)
    
    else:
        # в очереди не осталось места, заявка утеряна
        block2 += 1

if len(Mu) != 0:
    for i in range(len(Mu)):
        STP -= (Mu[0] - Time)

if len(R) != 0:
    for i in range(len(R)):
        SWT += Time - WS[i]

STP += SWT

In [4]:
print('Процент завок ушедших из системы по нетепеливости:', block1/All)
print('Процент заявок, заблокированых по нехватке места в системе:', block2/All)
print('Среднее время ожидания:', SWT/(All-block2))
print('Среднее время в системе:', STP/(All-block2))
print('Средняя длина очереди:', SR/All)
print('Среднее число заявок на приборах', S3/All)

Процент завок ушедших из системы по нетепеливости: 0.055451774653426406
Процент заявок, заблокированых по нехватке места в системе: 0.026841018582243633
Среднее время ожидания: 21.174646487540684
Среднее время в системе: 21.75839001954265
Средняя длина очереди: 0.6076098712024383
Среднее число заявок на приборах 5.6744666207845835
